In [47]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'../resources/fonts/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumGothic'}) # 폰트 설정
plt.rc('font', family='NanumGothic')
import seaborn as sns
import ast

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import TruncatedSVD

In [48]:
# 필요한 데이터 load
movie_path = r'C:\Users\subon\Upstage\ml-project-ml-pjt-2\src\resources\movies_metadata.csv'
rating_path = r'C:\Users\subon\Upstage\ml-project-ml-pjt-2\src\resources\ratings_small.csv'
credit_path = r'C:\Users\subon\Upstage\ml-project-ml-pjt-2\src\resources\credits.csv'
movie_dt = pd.read_csv(movie_path)
rating_dt = pd.read_csv(rating_path)
credit_dt = pd.read_csv(credit_path)

In [49]:
movie_dt.shape

(45466, 24)

In [50]:
#결측치 확인
movie_dt.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [51]:
rating_dt.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [52]:
credit_dt.isnull().sum()

cast    0
crew    0
id      0
dtype: int64

In [154]:
movie_dt.head(3)

,genres,id,original_language,popularity,release_date,status,title,vote_average,actor_names_x,director_names_x,actor_names_y,director_names_y,movieId
0,"Animation, Comedy, Family",862,en,21.946943,1995-10-30,Released,Toy Story,7.7,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter,1
1,"Adventure, Fantasy, Family",8844,en,17.015539,1995-12-15,Released,Jumanji,6.9,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston,2
2,"Romance, Comedy",15602,en,11.712900,1995-12-22,Released,Grumpier Old Men,6.5,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch,3


In [155]:
credit_dt.head(3)

,cast,crew,id,actor_names,director_names
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch


## 필요한 정보만 남기기

In [56]:
#필요한 컬럼만 선택
selected_columns = ['genres', 'id', 'original_language', 'popularity', 'release_date', 
                    'status', 'title', 'vote_average']
movie_dt= movie_dt[selected_columns]

In [57]:
#status가 released가 아닌 영화들 제외
movie_dt = movie_dt[movie_dt['status'] == 'Released'].dropna(subset=['status'])

#popularity 데이터타입 변환
movie_dt['popularity'] = movie_dt['popularity'].astype(float)

In [58]:
# genres 컬럼의 데이터 타입 및 일부 샘플 확인
print(movie_dt['genres'].dtype)
print(movie_dt['genres'].head(10))


object
0    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1    [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2    [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                       [{'id': 35, 'name': 'Comedy'}]
5    [{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...
6    [{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...
7    [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
8    [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
9    [{'id': 12, 'name': 'Adventure'}, {'id': 28, '...
Name: genres, dtype: object


In [59]:
#장르 추출
def extract_genre_names(genres_string):
    try:
        # 문자열일 때만 처리
        if isinstance(genres_string, str):
            genres_list = ast.literal_eval(genres_string)  # 문자열을 파이썬 객체로 변환
            genre_names = [genre['name'] for genre in genres_list]  # 이름만 추출
            return ', '.join(genre_names)
        else:
            return None  # 문자열이 아닌 경우 None 반환
    except (ValueError, SyntaxError):
        return None  # 변환에 실패하면 None 반환

# 'genres' 컬럼에 적용하여 'name' 리스트만 저장
movie_dt['genres'] = movie_dt['genres'].apply(extract_genre_names)

# 결과 확인
print(movie_dt['genres'].head())

0     Animation, Comedy, Family
1    Adventure, Fantasy, Family
2               Romance, Comedy
3        Comedy, Drama, Romance
4                        Comedy
Name: genres, dtype: object


In [60]:
#배우 추출
def extract_actor_names(cast_str):
    cast_list = ast.literal_eval(cast_str)
    actor_names = [cast_member['name'] for cast_member in cast_list]
    return ', '.join(actor_names)

credit_dt['actor_names'] = credit_dt['cast'].apply(extract_actor_names)

print(credit_dt[['actor_names']].head())

                                         actor_names
0  Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...
1  Robin Williams, Jonathan Hyde, Kirsten Dunst, ...
2  Walter Matthau, Jack Lemmon, Ann-Margret, Soph...
3  Whitney Houston, Angela Bassett, Loretta Devin...
4  Steve Martin, Diane Keaton, Martin Short, Kimb...


In [61]:
#감독 추출
def extract_director_names(crew_str):
    crew_list = ast.literal_eval(crew_str)
    director_names = [crew_member['name'] for crew_member in crew_list if crew_member['job'] == 'Director']
    return ', '.join(director_names)

credit_dt['director_names'] = credit_dt['crew'].apply(extract_director_names)

print(credit_dt[['director_names']].head())

    director_names
0    John Lasseter
1     Joe Johnston
2    Howard Deutch
3  Forest Whitaker
4    Charles Shyer


## 데이터 준비

In [68]:
#데이터 변환
movie_dt['id'] = movie_dt['id'].astype(str)
credit_dt['id'] = credit_dt['id'].astype(str)

In [69]:
movie_dt = pd.merge(movie_dt, credit_dt[['id', 'actor_names', 'director_names']], on='id', how='left')
movie_dt.head(5)

,genres,id,original_language,popularity,release_date,status,title,vote_average,actor_names_x,director_names_x,actor_names_y,director_names_y
0,"Animation, Comedy, Family",862,en,21.946943,1995-10-30,Released,Toy Story,7.7,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,"Adventure, Fantasy, Family",8844,en,17.015539,1995-12-15,Released,Jumanji,6.9,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston
2,"Romance, Comedy",15602,en,11.712900,1995-12-22,Released,Grumpier Old Men,6.5,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,"Comedy, Drama, Romance",31357,en,3.859495,1995-12-22,Released,Waiting to Exhale,6.1,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,Comedy,11862,en,8.387519,1995-02-10,Released,Father of the Bride Part II,5.7,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer


# 협업 필터링

## 모델 기반

#### <span style="color:blue">SVD 활용</span>

In [ ]:
# 사용자-영화 평점 매트릭스 만들기
user_movie_matrix = rating_dt.pivot(index='userId', columns='movieId', values='rating')

# 결측치는 0으로 대체
user_movie_matrix.fillna(0, inplace=True)

In [ ]:
# SVD 모델 정의
n_factors = 10  # 잠재 요인 개수
svd = TruncatedSVD(n_components=n_factors)

# 사용자-영화 매트릭스 분해
user_factors = svd.fit_transform(user_movie_matrix)
movie_factors = svd.components_.T

# 사용자-영화 평점 예측 (user_factors * movie_factors.T)
predicted_ratings = np.dot(user_factors, movie_factors.T)

In [219]:
from datetime import datetime

# 특정 사용자에게 추천할 영화 출력 함수
def recommend_movies(user_id, num_recommendations=5, bonus_weight=0.1):
    # 해당 사용자 ID에 대한 예측 평점 가져오기
    user_index = user_id - 1  # 0-based 인덱스 변환
    user_ratings = predicted_ratings[user_index]

    # 사용자가 이미 본 영화 확인
    watched_movies = user_movie_matrix.columns[user_movie_matrix.iloc[user_index, :] > 0].tolist()

    # 사용자가 평가한 영화의 개봉일을 가져옴 (movie_id가 movie_dt의 index+1에 존재하는 경우만)
    rated_movie_release_dates = []
    for movie_id in watched_movies:
        if not movie_dt.iloc[movie_id - 1].empty:  # movie_id는 index+1에 해당함
            release_date = movie_dt.iloc[movie_id - 1]['release_date']
            rated_movie_release_dates.append(pd.to_datetime(release_date))

    # 사용자가 보지 않은 영화 확인
    not_watched_movies = [movie_id for movie_id in user_movie_matrix.columns if movie_id not in watched_movies]

    # 사용자가 보지 않은 영화에 대해 예측된 평점 필터링 및 가중치 부여
    movie_scores_with_bonus = []

    for idx, predicted_rating in enumerate(user_ratings):
        movie_id = idx + 1  # movieId는 index+1에 해당
        movie_release_date = pd.to_datetime(movie_dt.iloc[idx]['release_date'])

        if pd.isna(movie_release_date) or len(rated_movie_release_dates) == 0:
            # 개봉일이 없는 영화 또는 평가한 영화가 없는 경우 가중치 없이 기본 점수만 반영
            proximity_bonus = 0
        else:
            # 사용자가 평가한 영화와 해당 영화의 개봉일 차이 계산
            date_diff = np.mean([abs((movie_release_date - rated_date).days) for rated_date in rated_movie_release_dates])

            # 날짜 차이에 따른 가중치 계산 (날짜 차이가 작을수록 가중치가 커짐)
            proximity_bonus = bonus_weight / (1 + date_diff)

        final_score = predicted_rating + proximity_bonus
        movie_scores_with_bonus.append((movie_id, final_score))

    top_recommendations = sorted(movie_scores_with_bonus, key=lambda x: x[1], reverse=True)[:num_recommendations]

    # 추천된 영화 ID로 영화 제목 반환
    for movie_id, _ in top_recommendations:
        movie_title = movie_dt.loc[movie_dt.index == movie_id - 1, 'title'].values  # movieId는 index+1에 해당
        if len(movie_title) > 0:
            print(f"추천 영화: {movie_title[0]}")
        else:
            print(f"추천 영화 ID {movie_id}에 해당하는 제목을 찾을 수 없습니다.")

# 실행
user_id = 332  
num_recommendations = 5  
recommend_movies(user_id, num_recommendations)

추천 영화: Blood, Guts, Bullets and Octane
추천 영화: It Happened One Night
추천 영화: Losing Isaiah
추천 영화: How To Make An American Quilt
추천 영화: Twelve Monkeys


#### <span style="color:blue">NCF 활용 </span>

In [ ]:
#!pip install deepctr-torch --user

In [179]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [196]:
rating_dt = pd.DataFrame(rating_dt)

In [198]:
train_df, test_df = train_test_split(rating_dt, test_size=0.2, random_state=42)

# Pytorch를 위해 장치 세팅 - GPU 사용, 불가 시 CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [199]:
# Step 1: NCF 모델 정의
class NCF(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=32, hidden_dim=64):
        super(NCF, self).__init__()
        # 임베딩
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)

        # Neural network layers
        self.fc1 = nn.Linear(embedding_dim * 2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc3 = nn.Linear(hidden_dim // 2, 1)
        
        self.relu = nn.ReLU()

    def forward(self, user_ids, item_ids):
        # 임베딩값 가져오기
        user_embeds = self.user_embedding(user_ids)
        item_embeds = self.item_embedding(item_ids)

        # 임베딩 값 연결
        x = torch.cat([user_embeds, item_embeds], dim=1)

        # neural network layer 통과시키기 
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [208]:
# Step 2: PyTorch에 사용할 데이터 준비
max_user_id = rating_df['userId'].max()
max_movie_id = rating_df['movieId'].max()

train_users = torch.tensor(train_df['userId'].values - 1, dtype=torch.long).to(device)
train_items = torch.tensor(train_df['movieId'].values - 1, dtype=torch.long).to(device)
train_ratings = torch.tensor(train_df['rating'].values, dtype=torch.float32).to(device)

test_users = torch.tensor(test_df['userId'].values - 1, dtype=torch.long).to(device)
test_items = torch.tensor(test_df['movieId'].values - 1, dtype=torch.long).to(device)
test_ratings = torch.tensor(test_df['rating'].values, dtype=torch.float32).to(device)

In [204]:
# Step 3: 초기화 및 모델 학습
embedding_dim = 32
hidden_dim = 64

model = NCF(num_users=num_users, num_items=num_items, embedding_dim=embedding_dim, hidden_dim=hidden_dim).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [205]:
# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()

    optimizer.zero_grad()
    
    #forward 패스
    predictions = model(train_users, train_items).squeeze()
    loss = criterion(predictions, train_ratings)

    # Backward pass, 최적화
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

Epoch 1/5, Loss: 14.225589752197266
Epoch 2/5, Loss: 13.969388961791992
Epoch 3/5, Loss: 13.722488403320312
Epoch 4/5, Loss: 13.484285354614258
Epoch 5/5, Loss: 13.254104614257812


In [206]:
# Step 4: 모델 평가 
model.eval()
with torch.no_grad():
    test_predictions = model(test_users, test_items).squeeze()
    test_loss = criterion(test_predictions, test_ratings)
    print(f"Test Loss: {test_loss.item()}")

Test Loss: 13.10325813293457


In [215]:
movie_dt['release_date'] = pd.to_datetime(movie_dt['release_date'], errors='coerce')  # 잘못된 날짜는 NaT로 처리

# 특정 사용자에게 영화 추천 (개봉일 근접도에 따른 가중치 부여)
def recommend_movies(user_id, num_recommendations=5, bonus_weight=0.1):
    max_movie_id = len(movie_dt)

    # 주어진 사용자의 모든 영화에 대한 예측 평점 생성
    user_tensor = torch.tensor([user_id - 1] * max_movie_id, dtype=torch.long).to(device)  
    item_tensor = torch.tensor(range(max_movie_id), dtype=torch.long).to(device)

    # 모델을 평가 모드로 전환하여 예측 수행
    model.eval()
    with torch.no_grad():
        predictions = model(user_tensor, item_tensor).squeeze().cpu().numpy()

    # 사용자가 평가한 영화 가져오기
    user_rated_movies = rating_dt[rating_dt['userId'] == user_id]['movieId'].values
    rated_movie_release_dates = movie_dt[movie_dt.index.isin(user_rated_movies - 1)]['release_date'].values

    # 개봉일 차이에 따른 근접도 가중치 계산
    movie_scores_with_bonus = []
    for idx, predicted_rating in enumerate(predictions):
        if idx >= len(movie_dt): 
            continue

        movie_id = movie_dt.iloc[idx]['id'] 
        movie_release_date = movie_dt.iloc[idx]['release_date']

        if pd.isna(movie_release_date):  # 개봉일이 없는 영화는 제외
            continue

        # 사용자가 평가한 영화들의 개봉일과 현재 영화의 개봉일 차이 계산
        date_diff = np.mean([abs((movie_release_date - rated_date).days) for rated_date in rated_movie_release_dates if not pd.isna(rated_date)])

        # 개봉일 차이에 따른 가중치 계산 (날짜 차이가 작을수록 더 높은 가중치 부여)
        proximity_bonus = 1 / (1 + date_diff) 

        total_score = predicted_rating + bonus_weight * proximity_bonus
        movie_scores_with_bonus.append((movie_id, total_score))

    top_recommendations = sorted(movie_scores_with_bonus, key=lambda x: x[1], reverse=True)[:num_recommendations]

    # 추천된 영화의 제목 가져오기
    top_movie_ids = [rec[0] for rec in top_recommendations]
    recommended_movies = movie_dt[movie_dt['id'].isin(top_movie_ids)]['title'].values[:num_recommendations]

    return recommended_movies


# 실행
user_id = 1 
num_recommendations = 5 
recommended_movies = recommend_movies(user_id, num_recommendations)
print(f"사용자 {user_id}에게 추천된 영화: {recommended_movies}")


Recommended Movies for User 1: ['This Christmas' 'The Last Letter' "Neptune's Daughter"
 'Flodder Does Manhattan!' 'Rebirth']


#### <span style="color:blue">GNNs 활용 </span>

## 메모리 기반

### 사용자 기반 협업 필터링 - 사용자 간의 유사도 비교

In [160]:
# 사용자-영화 평점 데이터를 데이터프레임으로 변환
rating_df = pd.DataFrame(rating_dt)
movie_df = pd.DataFrame(movie_dt)

In [171]:
user_movie_matrix = rating_dt.pivot_table(index='userId', columns='movieId', values='rating')

In [172]:
# 사용자 간 유사도 계산 (코사인 유사도)
user_similarity = cosine_similarity(user_movie_matrix.fillna(0))
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

In [163]:
# 사용자 간 유사도 출력
print("User Similarity Matrix:\n", user_similarity_df)

User Similarity Matrix:
 userId       1         2         3         4         5         6         7    \
userId                                                                         
1       1.000000  0.000000  0.000000  0.074482  0.016818  0.000000  0.083884   
2       0.000000  1.000000  0.124295  0.118821  0.103646  0.000000  0.212985   
3       0.000000  0.124295  1.000000  0.081640  0.151531  0.060691  0.154714   
4       0.074482  0.118821  0.081640  1.000000  0.130649  0.079648  0.319745   
5       0.016818  0.103646  0.151531  0.130649  1.000000  0.063796  0.095888   
...          ...       ...       ...       ...       ...       ...       ...   
667     0.000000  0.425462  0.124562  0.088735  0.058252  0.000000  0.232051   
668     0.000000  0.084646  0.124911  0.068483  0.042926  0.019563  0.058773   
669     0.062917  0.024140  0.080984  0.104309  0.038358  0.024583  0.073151   
670     0.000000  0.170595  0.136606  0.054512  0.062642  0.019465  0.096240   
671     0.01746

In [168]:
# 특정 사용자에게 영화 추천
def user_based_recommend(user_id, num_recommendations=5):
    # 현재 사용자와 다른 사용자 간의 유사도 가져오기
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)
    
    # 가장 유사한 사용자의 평점 데이터를 가져옴
    similar_users_ratings = user_movie_matrix.loc[similar_users.index]

    # 사용자가 이미 본 영화 제외
    user_ratings = user_movie_matrix.loc[user_id]
    watched_movies = user_ratings[user_ratings > 0].index

    # 다른 유사한 사용자들의 평균 평점 계산 (자신이 본 영화는 제외)
    movie_recommendations = similar_users_ratings.apply(lambda x: np.dot(similar_users, x) / similar_users.sum(), axis=0)
    movie_recommendations = movie_recommendations.drop(watched_movies)

    # 평점이 높은 순으로 영화 추천
    top_recommendations = movie_recommendations.sort_values(ascending=False).head(num_recommendations)
    
    # 영화 제목 출력
    for movie_id in top_recommendations.index:
        movie_title = movie_dt.loc[movie_dt.index == (movie_id - 1), 'title'].values
        if len(movie_title) > 0:
            print(f"추천 영화: {movie_title[0]}")

# 사용자 기반 협업 필터링 추천
user_id = 334  # 예시 사용자 ID
user_based_recommend(user_id, num_recommendations=5)

추천 영화: Jumanji
추천 영화: Grumpier Old Men
추천 영화: Waiting to Exhale
추천 영화: Father of the Bride Part II
추천 영화: Heat


### 아이템 기반 필터링 - 영화 간의 유사도 비교

In [174]:
# 사용자-아이템 평점 매트릭스 만들기
user_movie_matrix = rating_dt.pivot(index='userId', columns='movieId', values='rating')

# 결측치는 0으로 대체 (NaN 처리)
user_movie_matrix.fillna(0, inplace=True)

In [175]:
# 영화 간 코사인 유사도 계산
movie_similarity = cosine_similarity(user_movie_matrix.T)
movie_similarity_df = pd.DataFrame(movie_similarity, index=user_movie_matrix.columns, columns=user_movie_matrix.columns)

In [177]:
# 특정 사용자에게 아이템 기반 추천 함수 정의
def item_based_recommend(user_id, num_recommendations=5):
    # 사용자의 평점 가져오기
    user_ratings = user_movie_matrix.loc[user_id]
    
    # 사용자가 본 영화 중에서 평점이 높은 영화 선택
    high_rated_movies = user_ratings[user_ratings > 0].sort_values(ascending=False).index

    # 각 영화에 대해 유사한 영화 추천 (아이템 기반 협업 필터링)
    movie_scores = pd.Series(dtype=np.float64)
    
    for movie_id in high_rated_movies:
        # 해당 영화와 유사한 영화의 유사도 점수 가져오기
        similar_movies = movie_similarity_df[movie_id]
        similar_movies = similar_movies.drop(high_rated_movies)  # 사용자가 이미 본 영화는 제외
        
        # 유사도 점수를 누적
        movie_scores = movie_scores.add(similar_movies, fill_value=0)

    # 평점이 높은 순으로 영화 추천
    top_recommendations = movie_scores.sort_values(ascending=False).head(num_recommendations)

    # 영화 제목 출력
    for movie_id in top_recommendations.index:
        # movie_dt에서 영화 제목 가져오기 (movie_id와 index 매핑)
        movie_title = movie_dt.loc[movie_dt['id'] == movie_id, 'title'].values
        if len(movie_title) > 0:
            print(f"추천 영화: {movie_title[0]}")

# 아이템 기반 협업 필터링 추천 실행
user_id = 1  # 예시 사용자 ID
item_based_recommend(user_id, num_recommendations=5)

추천 영화: Superstar: The Karen Carpenter Story
추천 영화: Street Kings


# 내용 기반(Content Based) 필터링

# 시행착오 (무시)

In [76]:
# 빈 문자열 또는 공백만 있는 title을 찾는 코드
empty_titles = movie_dt[movie_dt['title'].str.strip() == '']

# 결과 출력
print(f"빈 제목을 가진 영화의 개수: {len(empty_titles)}")
print(empty_titles[['id', 'title']])

빈 제목을 가진 영화의 개수: 0
Empty DataFrame
Columns: [id, title]
Index: []


In [114]:
def recommend_movies_user_based(user_id, user_movie_matrix, user_similarity_df, movie_df, num_recommendations=5):
    # 해당 사용자와 유사한 사용자 목록 가져오기
    similar_users = user_similarity_df[user_id].drop(user_id).sort_values(ascending=False)

    # 유사한 사용자가 평가한 영화의 가중 평점 계산
    similar_user_ratings = user_movie_matrix.loc[similar_users.index]
    weighted_ratings = similar_users.dot(similar_user_ratings.fillna(0)) / similar_users.sum()

    # 추천할 영화 정렬 후 상위 추천
    recommendations = weighted_ratings.sort_values(ascending=False).head(num_recommendations)

    # 인덱스 타입을 맞춰주기 위해 movie_df의 'id'와 recommendations의 index를 둘 다 정수형으로 변환
    movie_df['id'] = movie_df['id'].astype(int)
    
    # movie_df와 recommendations 간의 공통된 영화 ID만 선택
    recommended_movies = movie_df[movie_df['id'].isin(recommendations.index.astype(int))]

    # 매칭된 영화 ID에 해당하는 예측 평점만 남기기
    matched_recommendations = recommendations[recommendations.index.isin(recommended_movies['id'])]

    # 예측 평점 추가
    recommended_movies['predicted_rating'] = matched_recommendations.values

    return recommended_movies[['id', 'title', 'predicted_rating']]

# 사용자 1에게 추천할 영화 찾기
user_id = 340
recommended_movies = recommend_movies_user_based(user_id, user_movie_matrix, user_similarity_df, movie_df)

print(f"Recommended movies for user {user_id}:\n{recommended_movies}")

Recommended movies for user 340:
       id                               title  predicted_rating
3382  593                             Solaris          2.991349
4018  318            The Million Dollar Hotel          2.792295
6388  296  Terminator 3: Rise of the Machines          2.443734
